<a href="https://colab.research.google.com/github/Crunza17/Colab.ChatBot/blob/main/MainApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/Proyecto"
VECTOR_STORE_DIR = f"{PROJECT_ROOT}/vector_store"

print(f"Ruta de la base de datos vectorial: {VECTOR_STORE_DIR}")

Mounted at /content/drive
Ruta de la base de datos vectorial: /content/drive/MyDrive/Proyecto/vector_store


In [ ]:
## Desinstalamos todo para empezar desde un estado completamente limpio.

# Instalamos versiones exactas y recientes que sabemos que funcionan juntas.
!pip install langchain langchain-core langchain-community
!pip install langchain-google-genai
!pip install sentence-transformers
!pip install pypdf
!pip install -U gradio
!pip install protobuf==3.20.3
!pip install chromadb==0.4.22
!pip install pysqlite3-binary==0.5.2

print("\n✅ ¡Instalación completada! Por favor, REINICIA LA SESIÓN para que los cambios surtan efecto.")

In [ ]:
# API Key guardada en Google Secrets
from google.colab import userdata
import os

try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ API Key de Google configurada correctamente.")
except Exception as e:
    print("⚠️ Error: No se pudo encontrar la API Key. Asegúrate de haberla guardado en los 'Secretos' con el nombre GOOGLE_API_KEY.")

✅ API Key de Google configurada correctamente.


In [ ]:
# Importamos todas las herramientas necesarias de LangChain y Gradio
import gradio as gr
import time
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

class ChatbotRAG:
    # NUEVO MÉTODO para formatear los documentos en un solo string
    def format_docs(self, docs):
        """Combina el page_content de varios documentos en un solo string."""
        return "\n\n---\n\n".join(doc.page_content for doc in docs)

    def __init__(self, vector_store_path):
        print("⏳ Inicializando el chatbot con la nueva arquitectura LCEL...")

        print("   - Cargando modelo de embeddings (Hugging Face)...")
        model_name = "sentence-transformers/all-MiniLM-L6-v2"
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)

        print(f"   - Cargando base de datos vectorial desde '{vector_store_path}'...")
        self.vector_store = Chroma(
            persist_directory=vector_store_path,
            embedding_function=self.embeddings,
            collection_name="documentos_quimica"
        )

        retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})

        print("   - Configurando el modelo de lenguaje de Google (Gemini)...")
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-pro-latest", # Usando el nombre del modelo más reciente y estable
            temperature=0.3,
            convert_system_message_to_human=True
        )

        prompt_template = """
        Eres un asistente amigable y servicial para estudiantes y maestros. Tu tarea es responder a las preguntas de los usuarios basándote únicamente en el contexto proporcionado.
        **Responde siempre en español, sin importar el idioma del contexto.** Si la respuesta no se encuentra en el contexto, di amablemente "Lo siento, no tengo información sobre eso en mis documentos."

        CONTEXTO:
        {context}

        PREGUNTA:
        {question}

        Respuesta útil:
        """
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        print("   - Creando la cadena de RAG con LCEL...")
        self.rag_chain = (
            # --- ¡AQUÍ ESTÁ LA MODIFICACIÓN CLAVE! ---
            # Ahora, el contexto se crea pasando la salida del retriever
            # a través de nuestra nueva función de formato.
            {"context": retriever | self.format_docs, "question": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )
        print("\n✅ ¡Chatbot RAG (LCEL) listo para funcionar!")

    def get_response(self, question):
        """Obtiene una respuesta del chatbot usando la nueva cadena LCEL."""
        try:
            result = self.rag_chain.invoke(question)
            return result
        except Exception as e:
            print(f"Error durante la invocación de la cadena: {e}")
            return "Lo siento, ha ocurrido un error al procesar tu pregunta."

# Creamos la instancia del chatbot (esta línea no cambia)
chatbot_instance_rag = ChatbotRAG(vector_store_path=VECTOR_STORE_DIR)

⏳ Inicializando el chatbot con la nueva arquitectura LCEL...
   - Cargando modelo de embeddings (Hugging Face)...
   - Cargando base de datos vectorial desde '/content/drive/MyDrive/Proyecto/vector_store'...


AttributeError: `np.float_` was removed in the NumPy 2.0 release. Use `np.float64` instead.

In [ ]:
# Celda de prueba en MainApp.ipynb

print("🔬 Realizando una prueba COMPLETA de la cadena RAG...")
test_question = "¿Que es molaridad?"

try:
    # Invocamos la cadena completa, exactamente como lo haría Gradio.
    final_response = chatbot_instance_rag.get_response(test_question)

    print("\n✅ ¡La cadena RAG se ejecutó exitosamente!")
    print("\nRespuesta generada por el LLM:")
    print("--------------------------------")
    print(final_response)
    print("--------------------------------")

    if "no tengo información" in final_response.lower():
         print("\n⚠️ ADVERTENCIA: La cadena funcionó, pero el LLM no encontró la respuesta en el contexto. Revisa el paso de formateo.")
    else:
        print("\n🎉 ¡Éxito total! La cadena está devolviendo contexto relevante.")

except Exception as e:
    print(f"\n❌ Ocurrió un error al invocar la cadena RAG: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
import gradio as gr

def chat_function_with_history(message, history):
    respuesta_bot = chatbot_instance_rag.get_response(message)

    for i in range(len(respuesta_bot)):
        time.sleep(0.02)
        yield respuesta_bot[: i+1]

chat_interface = gr.ChatInterface(
    fn=chat_function_with_history,
    title="🤖 Asistente Virtual de Química",
    description="Chatea conmigo sobre la carrera. Recuerdo nuestra conversación.",
    examples=[ee
        "¿Qué materias se ven en primer semestre?",
        "¿Que es molaridad?"
    ],

    chatbot=gr.Chatbot(
        avatar_images=(None, "https://i.imgur.com/2KmzC5E.png"),
        type="messages"
    ),

    theme=gr.themes.Soft(primary_hue="blue"),
)

print("🚀 Lanzando la interfaz de Gradio...")
chat_interface.launch(share=True, debug=True)